In [1]:
#| default_exp payment_providers


# payment providers

> Payment providers are the interfaces and implementations for payment methods.

In [2]:
#| export

from abc import ABC, abstractmethod
from datetime import datetime
from typing import Optional, TypedDict

from fastcore.utils import *
import fastcore.basics as fc
import os
import json
from l402.core import *
from l402.utils import *
import httpx
from pydantic import BaseModel


In [3]:
#| export

class PaymentProvider(ABC):
    """Base class for all payment providers"""
    supported_methods: list[str] = []  # Will be overridden by each provider
    
    @abstractmethod
    def pay(self, *args, **kwargs):
        pass


## Coinbase Provider

In [4]:
#| export

from cdp import *

class CoinbaseProvider(fc.BasicRepr):
    def __init__(self, wallet: Wallet,
                asset: str = 'usdc'):
        store_attr()
        self.supported_methods=["onchain"]
        self.chain = self.wallet.network_id

    # NOTE: in the fewsats example, we will return an async taskID
    def pay(self,
            amount: float,
            address: str,
            asset: str):
        # TODO, return a generic async task 
        return self.wallet.transfer(amount, asset, address).wait()

In [5]:
c = CoinbaseProvider(wallet=None)
c

CoinbaseProvider(wallet=None, chain='base-mainnet', asset='usdc', supported_methods=['onchain'])

In [6]:
#| export

class PaymentRequest(BaseModel):
    """Represents a payment request to get payment details from a payment provider"""
    offer_id: str
    payment_context_token: str
    payment_method: str
    chain: str = ""
    asset: str = ""

# class PaymentStatus:
#     PENDING = "pending"
#     COMPLETED = "completed"
#     FAILED = "failed"
#     EXPIRED = "expired"


In [7]:
r = httpx.get('http://localhost:8000/offers')
o = r.json()
r.status_code, o

(402,
 {'offers': [{'amount': 1,
    'currency': 'USD',
    'description': 'Purchase 1 credit for API access',
    'offer_id': 'ed40e2e3-3f30-46c3-b15b-f9d080e7d0e9',
    'payment_methods': ['onchain'],
    'title': '1 Credit Package',
    'type': 'one-time'}],
  'payment_context_token': 'f213d14b-b228-41a4-b72b-6c1e781f8f6c',
  'payment_request_url': 'http://localhost:8000/payment_request',
  'version': '0.2.2'})

In [8]:
data = {
    "offer_id": first(o['offers'])['offer_id'],
    "payment_method": 'onchain',
    "chain": "base-sepolia",
    "payment_context_token": o['payment_context_token']
    }
r = httpx.post(o['payment_request_url'], json=data)
r.status_code, r.text

(200,
 '{"expires_at":"2025-01-23T20:08:09.556058+00:00","offer_id":"ed40e2e3-3f30-46c3-b15b-f9d080e7d0e9","payment_request":{"address":"0x733f47d88ca38219dd09885532bBddDd9bcCAd67","chain":"base-sepolia","asset":"usdc"},"version":"0.2.2"}')

In [15]:
#| export

def get_payment_request(payment_request_url: str,
                        payment_context_token: str,
                        offer_id: str, 
                        payment_method: str, 
                        chain: str = "", 
                        asset: str = ""):
    data = {
        "offer_id": offer_id,
        "payment_method": payment_method,
        "chain": chain,
        "asset": asset,
        "payment_context_token": payment_context_token
    }
    r = httpx.post(payment_request_url, json=data)
    r.raise_for_status()
    return r.json()


In [16]:
r = get_payment_request(o['payment_request_url'], o['payment_context_token'], o['offers'][0]['offer_id'], 'onchain', 'base-sepolia', 'usdc')
r


http://localhost:8000/payment_request f213d14b-b228-41a4-b72b-6c1e781f8f6c ed40e2e3-3f30-46c3-b15b-f9d080e7d0e9 onchain base-sepolia usdc


{'expires_at': '2025-01-23T20:39:14.352877+00:00',
 'offer_id': 'ed40e2e3-3f30-46c3-b15b-f9d080e7d0e9',
 'payment_request': {'address': '0x32323E517Edfd5646eCc2Ff953e4087850994Dbe',
  'chain': 'base-sepolia',
  'asset': 'usdc'},
 'version': '0.2.2'}

In [20]:
#| export

class Client(fc.BasicRepr):
    def __init__(self, lightning_provider = None, 
                 credit_card_provider = None, 
                 onchain_provider = None):
        store_attr()
        self.lightning_provider = lightning_provider
        self.credit_card_provider = credit_card_provider
        self.onchain_provider = onchain_provider


    def pay(self, ofr: dict): # ofr is the l402 offers response dictionary
        "Pay for an offer"
        # this actually does 3 things
        # 1. Selects offer
        # 2. Gets payment request details
        # 3. Uses user-provided payment method
        
        ofr = L402Response(**ofr)
        if len(ofr.offers) != 1: raise ValueError("Only one offer is supported")
        o = first(ofr.offers)

        if 'onchain' in o.payment_methods and self.onchain_provider:
            r = get_payment_request(ofr.payment_request_url, ofr.payment_context_token, o.offer_id, 'onchain', self.onchain_provider.chain, self.onchain_provider.asset)

            return self.onchain_provider.pay(o.amount, r['payment_request']['address'], r['payment_request']['asset'])
            
        # elif 'lightning' in o.payment_methods and self.lightning_provider:
        # elif 'credit_card' in o.payment_methods and self.credit_card_provider:
        else:
            raise ValueError(f"No payment provider available for {ofr.offers[0].payment_methods}")


In [23]:
w = create_test_wallet(fund=False)
c = Client(onchain_provider=CoinbaseProvider(wallet=w, chain='base-sepolia', asset='usdc'))
c.pay(o)


ofr offers=[Offer(amount=1, currency='USD', description='Purchase 1 credit for API access', offer_id='ed40e2e3-3f30-46c3-b15b-f9d080e7d0e9', payment_methods=['onchain'], title='1 Credit Package', type='one-time')] payment_context_token='f213d14b-b228-41a4-b72b-6c1e781f8f6c' payment_request_url='http://localhost:8000/payment_request' version='0.2.2'
o amount=1 currency='USD' description='Purchase 1 credit for API access' offer_id='ed40e2e3-3f30-46c3-b15b-f9d080e7d0e9' payment_methods=['onchain'] title='1 Credit Package' type='one-time'
o.payment_methods ['onchain']
http://localhost:8000/payment_request f213d14b-b228-41a4-b72b-6c1e781f8f6c ed40e2e3-3f30-46c3-b15b-f9d080e7d0e9 onchain base-sepolia usdc


InsufficientFundsError: Insufficient funds: have 0, need 1.